In [1]:
from DipoleGroundState import MonoclinicLattice
import numpy as np
import scipy.sparse
import time
import random
import spinh 
import matplotlib.pyplot as plt

pi = np.pi
mB=9.274*10**(-24)
k=1.380*10**(-23)
NA=6.022*10**23
mu0=4*pi*10**(-7)
ErCl3 = MonoclinicLattice()
ErCl3.axes(9.57, 6.47, 7.84, 93.65*np.pi/180)
ErCl3.g_tensor(13.74, 0.75, 257*np.pi/180)
ErCl3.ion1_position(0.25, 0.1521, 0.25)
ErCl3.ion2_position(0.75, 0.8479, 0.75)

A_positions, A_labels = ErCl3.spherical_bravais_lattice(150,1,'A',1,'A')
B_positions, B_labels = ErCl3.spherical_bravais_lattice(150,1,'A',1,'B')
vertices = np.concatenate((A_positions,B_positions),axis=0)
ion_vector = ErCl3.position['1A']

factor = (mu0)/(4*pi)/((10**(-10))**3)
rx = vertices[:,0] - ion_vector[0]
ry = vertices[:,1] - ion_vector[1]
rz = vertices[:,2] - ion_vector[2]
rtot = np.sqrt(np.sum(np.power(vertices - ion_vector,2),axis=1))
mux = 0
muy = 0
muz = 0.5*mB*ErCl3.gz

Bx_each_ion = factor*((3*(muz*rz)*rx)/(rtot**5))
By_each_ion = factor*((3*(muz*rz)*ry)/(rtot**5))
Bz_each_ion = factor*(-(muz/rtot**3) + (3*(muz*rz)*rz)/(rtot**5))

def probabilty_flipped(T,Bx,By,Bz):
    boltzmann = 20.8368 #GHz/Kelvin
    p1 = np.exp(-spinh.groundsplitting(M_gnd,Bx,By,Bz)/(boltzmann*T))
    p2 = np.exp(-0)
    propability_flipping = p1/(p1+p2)
    return propability_flipping

def field(Bxs,Bys,Bzs,vertices,T,Bx_applied,By_applied,Bz_applied):   
    bias = -probabilty_flipped(T,Bx_applied,By_applied,Bz_applied)
    randoms_orientations = np.sign(np.random.rand(len(vertices))+bias)
#     M = rho*np.array([mux,muy,muz])*(1-2*probabilty_flipped(T,*B))
    Bx = np.sum(Bxs*randoms_orientations) + Bx_applied
    By = np.sum(Bys*randoms_orientations) + By_applied
    Bz = np.sum(Bzs*randoms_orientations) + Bz_applied
    return (Bx, By, Bz)

import numpy as np
import spinh

#Applied a universal rotation to my zeeman tensors calculated from the spiral
#As a result the direction of maximam splitting is now along the z axis
#Now it's in the same coordinate frame as everything above
#Note that this hamiltonian doesn't have 
# an axial tensor, so it's not completely consistent

alpha_gnd =   0.0*np.pi/180
beta_gnd  = -90.0*np.pi/180
gamma_gnd =-167.571*np.pi/180
gx_gnd    =  13.1
gy_gnd    =   0.0
gz_gnd    =   0.55
alpha_exc =   7.924*np.pi/180
beta_exc  = -92.722*np.pi/180
gamma_exc =-168.467*np.pi/180
gx_exc    =  13.0
gy_exc    =   0.8
gz_exc    =   0.9
rho = 2/(np.dot(np.cross(ErCl3.a,ErCl3.b),ErCl3.c))/((10**(-10))**3)
M_gnd = spinh.M(alpha_gnd, beta_gnd, gamma_gnd, gx_gnd, gy_gnd, gz_gnd)
M_exc = spinh.M(alpha_exc, beta_exc, gamma_exc, gx_exc, gy_exc, gz_exc)

Algorithm:
Specify some T
Set B0 for first iteration
Calculate M from given B0 and T

Caculate field at 500 lattice points using B, T and M
Calculate the average B field
Repeat using most recent B field and it's corresponding M

In [ ]:
rho = 2/(np.dot(np.cross(ErCl3.a,ErCl3.b),ErCl3.c))/((10**(-10))**3)

for T in np.linspace(0.05,1,21):
    t = time.time()
    B = [0,0,0.05]
    M = rho*np.array([mux,muy,muz])*(1-2*probabilty_flipped(T,*B))
    B_list = []
    M_list = []
    for i in range(100):
        Bcavity = mu0*M/3
        field_list = []
        for j in range(20000):
            field_list.append(field(Bx_each_ion,By_each_ion,Bz_each_ion,vertices,T,*B) + Bcavity)
        B = np.mean(field_list,axis=0)
        M = rho*np.array([mux,muy,muz])*(1-2*probabilty_flipped(T,*B))
        B_list.append(B[2])
        M_list.append(M[2])
    print(T,np.mean(B_list[-20:]),np.std(B_list[-20:]))
    plt.plot(B_list[:],label=str(T)+' K')
plt.legend()

In [90]:
def field_samples(T,Bx,By,Bz,num_samples):
    distribution = np.empty((num_samples,3))
    for i in range(sample_length):
        distribution[i] = field(Bx_each_ion,By_each_ion,Bz_each_ion,vertices,T,Bx,By,Bz)
    return distribution

def plot_fields(distributions):
    f, (ax1, ax2, ax3) = plt.subplots(1,3)
    for dist in distributions:
        ax1.hist(dist[:,0]*1,50,range=(-0.2,0.2))
        ax1.set_xlabel("$B_x$ at site (T)")
        ax2.hist(dist[:,1]*1,50,range=(-0.2,0.2))
        ax2.set_xlabel("$B_y$ at site (T)")
        ax3.hist(dist[:,2]*1,50,range=(-0.2,0.2))
        ax3.set_xlabel("$B_z$ at site (T)")
    plt.legend()
    plt.show()



I've did some field calculations at different probabilities, and discovered that when the probability of a flip is less than 0.05, the field distribution is non-gaussian. There are other distinct modes either side of the main peak.

In [ ]:
sample_length = 50000
distribution1 = field_samples(4.2,0,0,0,sample_length)
distribution2 = field_samples(0.26,0,0,0.107,sample_length)
distribution3 = field_samples(2,0,0,0.2,sample_length)

In [126]:
import importlib
importlib.reload(spinh)
# plot_fields([distribution1,distribution2,distribution3])
# trans, intensities = spinh.transitions_and_intensities(M_gnd,M_exc,distribution1)
spinh.synthetic_spectrum(M_gnd,M_exc,distribution1,4.2)
spinh.synthetic_spectrum(M_gnd,M_exc,distribution2,0.26)
spinh.synthetic_spectrum(M_gnd,M_exc,distribution3,2)